In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchsummary import summary


In [15]:
# encoding components
resnet = models.resnet18(pretrained=True)
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
modules = list(resnet.children())[:-1]      # delete the last fc layer.
model = nn.Sequential(*modules)

In [18]:
summary(model.cuda(), (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 14, 14]           3,136
       BatchNorm2d-2           [-1, 64, 14, 14]             128
              ReLU-3           [-1, 64, 14, 14]               0
         MaxPool2d-4             [-1, 64, 7, 7]               0
            Conv2d-5             [-1, 64, 7, 7]          36,864
       BatchNorm2d-6             [-1, 64, 7, 7]             128
              ReLU-7             [-1, 64, 7, 7]               0
            Conv2d-8             [-1, 64, 7, 7]          36,864
       BatchNorm2d-9             [-1, 64, 7, 7]             128
             ReLU-10             [-1, 64, 7, 7]               0
       BasicBlock-11             [-1, 64, 7, 7]               0
           Conv2d-12             [-1, 64, 7, 7]          36,864
      BatchNorm2d-13             [-1, 64, 7, 7]             128
             ReLU-14             [-1, 6

In [107]:

    

class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample):
        super().__init__()
        if downsample:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
#                 nn.BatchNorm2d(out_channels)
            )
        else:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
            self.shortcut = nn.Sequential()

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
#         self.bn1 = nn.BatchNorm2d(out_channels)
#         self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, input):
        shortcut = self.shortcut(input)
#         input = nn.ReLU()(self.bn1(self.conv1(input)))
#         input = nn.ReLU()(self.bn2(self.conv2(input)))
        input = nn.ReLU()(self.conv1(input))
        input = nn.ReLU()(self.conv2(input))        
        input = input + shortcut
        return nn.ReLU()(input)
    
class ResNetEncoder(nn.Module):
    def __init__(self, in_channels, resblock):
        super().__init__()

        self.layer0 = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=5, stride=1, padding=0),
#             nn.MaxPool2d(kernel_size=2, stride=2, padding=1
#             nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.layer1 = nn.Sequential(
            resblock(32,64, downsample=True),
            resblock(64, 64, downsample=False)
        )


        self.layer2 = nn.Sequential(
            resblock(64, 128, downsample=True),
            resblock(128, 128, downsample=False)
        )
        
        self.layer3 = nn.Sequential(
            resblock(128, 256, downsample=True),
            resblock(256, 256, downsample=False)
        )

    def forward(self, input):
        input = self.layer0(input)
        input = self.layer1(input)
        input = self.layer2(input)
        input = self.layer3(input)
#         input = self.layer4(input)
#         input = self.gap(input)
#         input = torch.flatten(input)
#         input = self.fc(input)

        return input

In [108]:
model = ResNetEncoder(in_channels=1, resblock= ResBlock)
# resnet18.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
summary(model.cuda(), (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 24, 24]             832
              ReLU-2           [-1, 32, 24, 24]               0
            Conv2d-3           [-1, 64, 12, 12]           2,112
            Conv2d-4           [-1, 64, 12, 12]          18,496
            Conv2d-5           [-1, 64, 12, 12]          36,928
          ResBlock-6           [-1, 64, 12, 12]               0
            Conv2d-7           [-1, 64, 12, 12]          36,928
            Conv2d-8           [-1, 64, 12, 12]          36,928
          ResBlock-9           [-1, 64, 12, 12]               0
           Conv2d-10            [-1, 128, 6, 6]           8,320
           Conv2d-11            [-1, 128, 6, 6]          73,856
           Conv2d-12            [-1, 128, 6, 6]         147,584
         ResBlock-13            [-1, 128, 6, 6]               0
           Conv2d-14            [-1, 12

In [74]:

class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample):
        super().__init__()
        if downsample:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
#                 nn.BatchNorm2d(out_channels)
            )
        else:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
            self.shortcut = nn.Sequential()

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
#         self.bn1 = nn.BatchNorm2d(out_channels)
#         self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, input):
        shortcut = self.shortcut(input)
#         input = nn.ReLU()(self.bn1(self.conv1(input)))
#         input = nn.ReLU()(self.bn2(self.conv2(input)))
        input = nn.ReLU()(self.conv1(input))
        input = nn.ReLU()(self.conv2(input))        
        input = input + shortcut
        return nn.ReLU()(input)


class ResNet(nn.Module):
    def __init__(self, in_channels, resblock, outputs=10):
        super().__init__()

        self.layer0 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=5, stride=1, padding=0),
#             nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
#             nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.layer1 = nn.Sequential(
            resblock(64, 128, downsample=True),
            resblock(128, 128, downsample=False)
        )


        self.layer2 = nn.Sequential(
            resblock(128, 256, downsample=True),
            resblock(256, 256, downsample=False)
        )
        
        self.layer3 = nn.Sequential(
            resblock(256, 512, downsample=True),
            resblock(512, 512, downsample=False)
        )

        self.gap = torch.nn.AdaptiveAvgPool2d(1)
        self.fc = torch.nn.Linear(1024, outputs)

    def forward(self, input):
        input = self.layer0(input)
        input = self.layer1(input)
        input = self.layer2(input)
        input = self.layer3(input)
        input = self.gap(input)
        input = torch.flatten(input)
        input = self.fc(input)

        return input


In [75]:
model = ResNet(in_channels=1, resblock= ResBlock, outputs=10)
# resnet18.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
summary(model.cuda(), (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 24, 24]           1,664
              ReLU-2           [-1, 64, 24, 24]               0
            Conv2d-3          [-1, 128, 12, 12]           8,320
            Conv2d-4          [-1, 128, 12, 12]          73,856
            Conv2d-5          [-1, 128, 12, 12]         147,584
          ResBlock-6          [-1, 128, 12, 12]               0
            Conv2d-7          [-1, 128, 12, 12]         147,584
            Conv2d-8          [-1, 128, 12, 12]         147,584
          ResBlock-9          [-1, 128, 12, 12]               0
           Conv2d-10            [-1, 256, 6, 6]          33,024
           Conv2d-11            [-1, 256, 6, 6]         295,168
           Conv2d-12            [-1, 256, 6, 6]         590,080
         ResBlock-13            [-1, 256, 6, 6]               0
           Conv2d-14            [-1, 25

In [100]:
256*3*3/8

288.0

In [58]:
512*6*6/8

2304.0

In [28]:
256*6*6/8

1152.0

In [18]:
64*12*12/8

1152.0

In [50]:
128/8

16.0

In [7]:
model

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con